## dependencies

In [ ]:
import napari
from napari.layers import Image, Tracks
from iohub import open_ome_zarr
import dask.array as da
import pandas as pd
import os
from pathlib import Path
from tifffile import imread

import numpy as np

from ultrack.utils import labels_to_contours
from ultrack.tracks import inv_tracks_df_forest


## Loading data

In [ ]:
#*************************************************
path = '<USER DEFINED PATH>/3_segmentation/membrane/membrane_masks.zarr'
path_raw = '<USER DEFINED PATH>/2_deconvolution_and_registration/deconvolved_and_registered.zarr'

Tmax = 500    
working_directory = '.'       
flag_closedGaps = ''
#*************************************************

#Load CellPose labels 
ds_cellpose = open_ome_zarr(path)
labels = da.from_zarr(ds_cellpose["0/2/0"].data)[:Tmax,0]

#Load raw 
ds_raw = open_ome_zarr(path_raw)  #ds_raw.print_tree()
raw_cyto = da.from_zarr(ds_raw['0/2/0'].data)[:Tmax,0]
raw_mem = da.from_zarr(ds_raw['0/2/0'].data)[:Tmax,1]

#Load Tracking results
tracking_files = []
for f in os.listdir(working_directory):
    if f.endswith("tracks" + flag_closedGaps + ".csv"):
        tracking_files.append(f)
if len(tracking_files)==1:
    print('saved_tracks loaded')
    f = tracking_files[0]
    saved_tracks = pd.read_csv(working_directory+f)
else:
    del saved_tracks
    print("Error: multile _tracks.csv files in dir")

segments = da.from_zarr(working_directory + 'segments' + flag_closedGaps + '/')
graph = inv_tracks_df_forest(saved_tracks)

## Division

In [ ]:
graph

In [ ]:
#******************
FOLLOW = 42  #[2 6 10 11 19 (20) 29 33 42! 54 57]
t = 0
z_slice  = saved_tracks[(saved_tracks['track_id']==FOLLOW) & (saved_tracks['t']==0)].z.values[0]
scale = (0.25, 0.108 ,0.108)
#******************

viewer = napari.Viewer()

# Cytoplasm
layer_cyto = viewer.add_image(raw_cyto,scale=scale,colormap='magenta',contrast_limits=(0.00, 0.13),name='raw_nucleus')
layer_cyto.opacity = 1
layer_cyto._auto_contrast_source
layer_cyto.refresh()
layer_cyto.visible = True

# Membrane
layer_mem = viewer.add_image(raw_mem, gamma=1,scale=scale,colormap='cyan',contrast_limits=(0, 0.4),name='raw_membrane')
layer_mem.opacity = 0.58
layer_mem.visible = True

# Ultrack segments
layer_seg = viewer.add_labels(segments,scale=scale, name="segments")#,color={0:'black', FOLLOW:(1,1,0,1)})
layer_seg.contour = 7   # 0 by default
layer_seg.opacity = 1
layer_seg.visible = True

#Ultrack - filter dataframe/graph
df = saved_tracks
filtered_df = df[
    (df['track_id'] == FOLLOW) | (df['parent_track_id'] == FOLLOW)
]
children_id = np.unique(filtered_df['track_id'].values)
children_id = np.delete(children_id, np.where(children_id==FOLLOW))
print('children',children_id)

filtered_graph = graph.copy()
for k in graph.keys():
    if graph[k] != FOLLOW:
        filtered_graph.pop(k)
print('filtered graph',filtered_graph)


# Ultrack segments
layer_child1 = viewer.add_labels(segments,scale=scale, name="child1")#,color={0:'black', children_id[0]:(0,0,1, 1)}) #purple
layer_child1.contour = 7   # 0 by default
layer_child1.opacity = 1
layer_child1.selected_label = children_id[0]
layer_child1.show_selected_label = True
layer_child1.visible = True

layer_child2 = viewer.add_labels(segments,scale=scale, name="child2")#,color={0:'black', children_id[1]:(1, 0.65, 0, 1)})
layer_child2.contour = 7   # 0 by default
layer_child2.opacity = 1
layer_child2.selected_label = children_id[1]
layer_child2.show_selected_label = True
layer_child2.visible = True

#Ultrack - tracks
layer_tracks = viewer.add_tracks(
    filtered_df[['track_id','t','z', 'y', 'x']],
    scale=scale,
    name = "tracks",
    visible=True,
    tail_length = 23, 
    tail_width = 10,
    colormap = 'twilight',
    graph=filtered_graph,
)


t_afterdiv = filtered_df[filtered_df['track_id']==children_id[0]]['t'].values.min()

viewer.dims.set_current_step(0, t_afterdiv)
viewer.dims.set_current_step(1, z_slice)
viewer.scale_bar.visible = True

#Cell 42
viewer.camera.center = (0, 32.2, 57.7)
viewer.camera.zoom = 52

viewer.window.resize(1400, 1000)

layer_seg.selected_label = FOLLOW
layer_seg.show_selected_label = True



# #CallBack 
maxTime_for_FOLLOW = saved_tracks[(saved_tracks['track_id']==FOLLOW)].t.max() 

sliders_pos_global = viewer.dims.current_step
def cb_func(event):
    global sliders_pos_global

    cur_time = viewer.dims.current_step[0]          #get slider T
    # print('cur_time',cur_time)

    new_pos = event.source.current_step
    if new_pos[0] != sliders_pos_global[0]:     #time changed
        sliders_pos_global = new_pos

    if cur_time <= maxTime_for_FOLLOW:
        track_z = saved_tracks[(saved_tracks['track_id']==FOLLOW) & (saved_tracks['t']==cur_time)].z.values[0] 
    else:
        track_z = saved_tracks[(saved_tracks['track_id']==children_id[1]) & (saved_tracks['t']==cur_time)].z.values[0] 
                    
                                                    # get track Z
    viewer.dims.set_point(1,track_z*scale[0])        # set slider Z

viewer.dims.events.current_step.connect(cb_func)

In [ ]:
viewer.camera.zoom

#### Saving screenshots

In [ ]:
FIG_DIR = Path('./500_frames/')

layer_cyto.gamma = 1.25
layer_cyto.contrast_limints = (0, 0.20)

t_vec = [420, 423, 425, 434]
for t in t_vec:
    print('time',t)
    
    viewer.dims.set_current_step(0, t)

    filename = "neuromast_events_div_frame" + str(t) + ".png"
    viewer.screenshot(FIG_DIR / filename)

#### Animation

Doesn't work yet, because some residual element in the viewer cannot be pickled. So distill the viewer and see what breaks the animation


In [ ]:
from napari_animation import Animation
FIG_DIR = Path('./500_frames/')

animation = Animation(viewer)

viewer.dims.set_point(0, 400)
animation.capture_keyframe()

viewer.dims.set_point(0, 440)
animation.capture_keyframe(40)



video_path = FIG_DIR / "neuromast_events_division_video_v1.mp4"
animation.animate(video_path, fps=10)



## Cell Death

In [ ]:
#******************
FOLLOW = 25
t = 4
z_slice  = saved_tracks[(saved_tracks['track_id']==FOLLOW) & (saved_tracks['t']==0)].z.values[0]
scale = (0.25, 0.108 ,0.108)
#******************

viewer = napari.Viewer()

# Cytoplasm
layer_cyto = viewer.add_image(raw_cyto,scale=scale,colormap='magenta',contrast_limits=(0.00, 0.13),name='raw_nucleus')
layer_cyto.opacity = 1
layer_cyto._auto_contrast_source
layer_cyto.refresh()
layer_cyto.visible = True

# Membrane
layer_mem = viewer.add_image(raw_mem, gamma=1,scale=scale,colormap='cyan',contrast_limits=(0, 0.4),name='raw_membrane')
layer_mem.opacity = 0.58
layer_mem.visible = True

# Ultrack segments
layer_seg = viewer.add_labels(segments,scale=scale, name="segments")#,color={FOLLOW:'black', FOLLOW:(0,1,0,1)})
layer_seg.contour = 7   # 0 by default
layer_seg.opacity = 1
layer_seg.visible = True
layer_seg.selected_label = FOLLOW
layer_seg.show_selected_label = True

#Ultrack - filter dataframe/graph
df = saved_tracks
filtered_df = df[
    df['track_id'] == FOLLOW
]

#Ultrack - tracks
layer_tracks = viewer.add_tracks(
    filtered_df[['track_id','t','z', 'y', 'x']],
    scale=scale,
    name = "tracks",
    visible=True,
    tail_length = 23, 
    tail_width = 10,
    colormap = 'twilight',
    # graph=filtered_graph,
)

time_of_death = saved_tracks[(saved_tracks['track_id']==FOLLOW)].t.max() 
print('time of death',time_of_death)
viewer.dims.set_current_step(0, time_of_death)
viewer.dims.set_current_step(1, z_slice)
viewer.scale_bar.visible = True

#Full FOV
# viewer.camera.center = (0, 29.18, 34.58)
# viewer.camera.zoom = 15.27
#follow = 28
viewer.camera.center = (0, 47, 62)
viewer.camera.zoom = 43


viewer.window.resize(1400, 1000)
layer_cyto.gamma = 0.74

#CallBack 
maxTime_for_FOLLOW = saved_tracks[(saved_tracks['track_id']==FOLLOW)].t.max() 

sliders_pos_global = viewer.dims.current_step
def cb_func(event):
    global sliders_pos_global

    cur_time = viewer.dims.current_step[0]          #get slider T
    # print('cur_time',cur_time)

    new_pos = event.source.current_step
    if new_pos[0] != sliders_pos_global[0]:     #time changed
        # layer.data = ultrack_layer[new_pos[0]]
        # print('time changed!')
        sliders_pos_global = new_pos


    #closest timepoint
    one_cell = saved_tracks[(saved_tracks['track_id']==FOLLOW)]
    track_z = one_cell.iloc[(one_cell['t'] - cur_time).abs().argsort()[:3]].z.values[0]

    # print('track_z',track_z)                        
                                                    # get track Z
    viewer.dims.set_point(1,track_z*scale[0])        # set slider Z

    ### print('time:',cur_time,'z:',cur_z,'track_z',track_z)  #print T

viewer.dims.events.current_step.connect(cb_func)


In [ ]:
viewer.camera.center

#### Saving screenshots

In [ ]:
FIG_DIR = Path('./500_frames/')

t_vec = [255, 258, 260, 266]
for id,t in enumerate(t_vec):
    print('time',t)

    viewer.dims.set_current_step(0, t)

    if id<=1:
        layer_tracks.visible = True
    else:
        layer_tracks.visible = False

    filename = "neuromast_events_death_frame" + str(t) + ".png"
    viewer.screenshot(FIG_DIR / filename)


#### Animation

In [ ]:
from napari_animation import Animation
FIG_DIR = Path('./500_frames/')




animation = Animation(viewer)

viewer.dims.set_point(0, 230)
animation.capture_keyframe()

viewer.dims.set_point(0, 280)
animation.capture_keyframe(40)



video_path = FIG_DIR / "neuromast_events_death_video_v1.mp4"
animation.animate(video_path, fps=10)



## Movement

In [ ]:
#******************
FOLLOW = 42

t = 10
z_slice  = saved_tracks[(saved_tracks['track_id']==FOLLOW) & (saved_tracks['t']==t)].z.values[0]
scale = (0.25, 0.108 ,0.108)
#******************

viewer = napari.Viewer()

# Cytoplasm
layer_cyto = viewer.add_image(raw_cyto,scale=scale,colormap='magenta',contrast_limits=(0.00, 0.13),name='raw_nucleus')
layer_cyto.opacity = 1
layer_cyto._auto_contrast_source
layer_cyto.refresh()
layer_cyto.visible = True

# Membrane
layer_mem = viewer.add_image(raw_mem, gamma=1,scale=scale,colormap='cyan',contrast_limits=(0, 0.4),name='raw_membrane')
layer_mem.opacity = 0.58
layer_mem.visible = True

# Ultrack segments
layer_seg = viewer.add_labels(segments,scale=scale, name="segments")#,color={FOLLOW:'black', FOLLOW:(1,0,0,1)})
layer_seg.contour = 7   # 0 by default
layer_seg.opacity = 1
layer_seg.visible = True

#Ultrack - filter dataframe/graph
df = saved_tracks
filtered_df = df[
    (df['track_id'] == FOLLOW)
]
# filtered_df = df

#Ultrack - tracks
layer_tracks = viewer.add_tracks(
    filtered_df[['track_id','t','z', 'y', 'x']],
    scale=scale,
    name = "tracks",
    visible=True,
    tail_length = 23, 
    tail_width = 10,
    colormap = 'twilight',
    # graph=filtered_graph,
)


viewer.dims.set_current_step(0, t)
viewer.dims.set_current_step(1, z_slice)
viewer.scale_bar.visible = True

#Full FOV
# viewer.camera.center = (0, 29.18, 34.58)
# viewer.camera.zoom = 15.27
#Cell 52
viewer.camera.center = (0, 11.4, 24)
viewer.camera.zoom = 45


viewer.window.resize(1400, 1000)


layer_cyto.gamma = 0.74
layer_cyto.contrast_limits = (0.01, 0.23)

layer_seg.selected_label = FOLLOW
layer_seg.show_selected_label = True

#### Saving screenshots

In [ ]:
# FIG_DIR = Path('./500_frames/')

# # layer_cyto.gamma = 0.74
# # layer_cyto.contrast_limits = (0.01, 0.1)


# # t_vec = [310, 320, 330, 340, 350, 360, 370]
# # t_vec = [25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
# t_vec = [45,50,55,70]
# for id,t in enumerate(t_vec):
#     print('time',t)

#     viewer.dims.set_current_step(0, t)

#     # if id<=1:
#     #     layer_tracks.visible = True
#     # else:
#     #     layer_tracks.visible = False

#     filename = "neuromast_events_move_frame" + str(t) + ".png"
#     viewer.screenshot(FIG_DIR / filename)


#### Animation

In [ ]:
from napari_animation import Animation
FIG_DIR = Path('./500_frames/')

animation = Animation(viewer)

viewer.dims.set_point(0, 10)
animation.capture_keyframe()

viewer.dims.set_point(0, 100)
animation.capture_keyframe(40)

video_path = FIG_DIR / "neuromast_events_video_v1.mp4"
animation.animate(video_path, fps=10)

## Cellpose mistakes

In [ ]:
#******************
FOLLOW = 7  #divides: [9 17 21 41 46 51], dies: [28]
t = 16
z_slice  = 30
scale = (0.25, 0.108 ,0.108)
#******************

viewer = napari.Viewer()

# Cytoplasm
layer_cyto = viewer.add_image(raw_cyto,scale=scale,colormap='magenta',contrast_limits=(0.00, 0.13),name='raw_nucleus')
layer_cyto.opacity = 1
layer_cyto._auto_contrast_source
layer_cyto.refresh()
layer_cyto.visible = True

# Membrane
layer_mem = viewer.add_image(raw_mem, gamma=1,scale=scale,colormap='cyan',contrast_limits=(0, 0.4),name='raw_membrane')
layer_mem.opacity = 0.58
layer_mem.visible = True

# CellPose
layer_cp = viewer.add_labels(labels,scale=scale, name="cellpose")
layer_cp.visible = True
layer_cp.opacity = 0.8

# Ultrack segments
layer_seg = viewer.add_labels(segments,scale=scale, name="segments") #,color={0:'black', FOLLOW:(1,1,0,1)}
layer_seg.contour = 0   # 0 by default
layer_seg.opacity = .8
layer_seg.visible = True

# #Ultrack - filter dataframe/graph
# df = saved_tracks
# filtered_df = df[
#     (df['track_id'] == FOLLOW)
# ]
# # filtered_df = df

# #Ultrack - tracks
# layer_tracks = viewer.add_tracks(
#     filtered_df[['track_id','t','z', 'y', 'x']],
#     scale=scale,
#     name = "tracks",
#     visible=True,
#     tail_length = 23, 
#     tail_width = 10,
#     colormap = 'twilight',
#     # graph=filtered_graph,
# )


viewer.dims.set_current_step(0, t)
viewer.dims.set_current_step(1, z_slice)
viewer.scale_bar.visible = True

#Full FOV
viewer.camera.center = (0, 29.18, 34.58)
viewer.camera.zoom = 15.27
#Cell 52
# viewer.camera.center = (0, 11.4, 24)
# viewer.camera.zoom = 45


viewer.window.resize(1400, 1000)


layer_cyto.gamma = 0.74
layer_cyto.contrast_limits = (0.01, 0.23)

layer_seg.selected_label = FOLLOW
layer_seg.show_selected_label = True

In [ ]:
viewer = napari.Viewer()

# CellPose
layer_cp = viewer.add_labels(labels,scale=scale, name="cellpose")
layer_cp.visible = True
layer_cp.opacity = 0.8

# Ultrack segments
layer_seg = viewer.add_labels(segments,scale=scale, name="segments") #,color={0:'black', FOLLOW:(1,1,0,1)}
layer_seg.contour = 0   # 0 by default
layer_seg.opacity = .8
layer_seg.visible = True

#### Saving screenshots

In [ ]:
viewer.camera.center

In [ ]:
viewer.dims.set_current_step(0, 93)  #time
viewer.dims.set_current_step(1, 33)  #z
viewer.camera.center = (0,36,70)
viewer.camera.zoom = 63

layer_cyto.contrast_limits = (0, .13)
layer_cyto.gamma = 1
layer_seg.selected_label = 7
layer_seg.show_selected_label = False
layer_mem.contrast_limits = (0, .40)
layer_mem.gamma = 1


FIG_DIR = Path('./500_frames/')

# viewer.dims.set_current_step(1, 31)  #z

layer_seg.visible = False
layer_cp.visible = False
viewer.screenshot(FIG_DIR / "neuromast_cellpose_mistakes_label34_v0.png")

# viewer.dims.set_current_step(1, 33)  #z

layer_seg.visible = False
layer_cp.visible = True
viewer.screenshot(FIG_DIR / "neuromast_cellpose_mistakes_label34_v1.png")

layer_seg.visible = True
layer_cp.visible = False
viewer.screenshot(FIG_DIR / "neuromast_cellpose_mistakes_label34_v2.png")


In [ ]:
viewer.dims.set_current_step(0, 269)  #time
viewer.dims.set_current_step(1, 33)  #z
viewer.camera.center = (0,27.5 ,71)
viewer.camera.zoom = 88

layer_cyto.contrast_limits = (0, .13)
layer_cyto.gamma = 1
layer_seg.selected_label = 7
layer_seg.show_selected_label = False
layer_mem.contrast_limits = (0, .27)


FIG_DIR = Path('./500_frames/')

layer_seg.visible = False
layer_cp.visible = False
viewer.screenshot(FIG_DIR / "neuromast_cellpose_mistakes_label7_v0.png")

layer_seg.visible = False
layer_cp.visible = True
viewer.screenshot(FIG_DIR / "neuromast_cellpose_mistakes_label7_v1.png")

layer_seg.visible = True
layer_cp.visible = False
viewer.screenshot(FIG_DIR / "neuromast_cellpose_mistakes_label7_v2.png")